In [1]:
# TD Ameritrade OAuth 2.0 authorization script
# This python script opens a browser,
# invokes a TD Ameritrade OAuth server
# with a url to receive an authorization code.
# Submits TD Ameritrade login information to receive tokens.
# Streams TD Ameritrade quote data via a websocket.
# https://github.com/2020dataanalysis/OAuth2.git
# Stream.ipynb
# 06/25/2020
# Author Sam Portillo

# References:
#   https://oauth.net/2/
#   https://developer.tdameritrade.com/


# conda activate TDA
# conda install -n myenv scipy
# conda install -n myenv pip
# conda activate myenv
# pip <pip_subcommand>
# ipython kernel install --name TDA --user
# or
# python -m ipykernel install --user
# conda install -c metaperl splinter  This did not work → Anaconda propt → pip install splinter

In [2]:
# # Request Post Access Token - Using the ‘authorization_code’ grant type

# import requests, time, urllib
# from splinter import Browser
# from selenium import webdriver
# from credentials import client_id, username, password, redirect_uri
# # Chrome, Help, About Google Chrome → version needs to match driver.
# executable_path = {'executable_path': r'C:\Z\drivers\83\chromedriver.exe' }
# browser = Browser('chrome', **executable_path, headless = False )
# # Build OAuth 2.0 url
# method = 'GET'
# url = 'https://auth.tdameritrade.com/auth?'
# payload = {'response_type':'code', 'redirect_uri':redirect_uri, 'client_id':client_id + '@AMER.OAUTHAP'}
# oauth_url = requests.Request(method, url, params = payload).prepare()
# browser.visit(oauth_url.url)
# # Fill out each element in the form
# browser.find_by_id("username").first.fill(username)
# browser.find_by_id("password").first.fill(password)
# browser.find_by_id("accept").first.click()

# #time.sleep(3)
# #browser.find_by_id("accept_pre").first.click()

# # Need time to manually enter TDA text key
# time.sleep(30)

# # Click to 'Continue'
# browser.find_by_id("accept").first.click()
# time.sleep(1)
# authorization_code = urllib.parse.unquote(browser.url.split('code=')[1])
# browser.quit()

# # Request Post Access Token - Using the ‘authorization_code’ grant type
# url = r'https://api.tdameritrade.com/v1/oauth2/token'
# headers = {'Content-Type':"application/x-www-form-urlencoded"}
# payload = {
#             'grant_type':'authorization_code',
#             'access_type':'offline',
#             'code':authorization_code,
#             'client_id':client_id,
#             'redirect_uri':redirect_uri
#           }

# response = requests.post(url, headers = headers, data = payload)

# # Convert python object into a JSON string
# tokens = response.json()
# print(tokens)

In [3]:
# Request Post Access Token - Using the refresh token
import requests, urllib
from credentials import client_id, refresh_token
url = r'https://api.tdameritrade.com/v1/oauth2/token'
headers = {'Content-Type':"application/x-www-form-urlencoded"}
payload = {
            'grant_type':'refresh_token',
            'refresh_token':refresh_token,
            'client_id':client_id
          }

response = requests.post(url, headers = headers, data = payload)

# Convert python object into a JSON string
tokens = response.json()
#print(tokens)

# Get User Principals

In [4]:
# Get User Principals
# <Response [200]> → Success
endpoint = 'https://api.tdameritrade.com/v1/userprincipals'
headers = {'Authorization':'Bearer {}'.format(tokens['access_token'])}
params = {'fields': 'streamerSubscriptionKeys,streamerConnectionInfo'}
content = requests.get( url = endpoint, params = params, headers = headers )
content

<Response [200]>

In [5]:
import dateutil.parser
import datetime
import json

def unix_time_millis(dt):
    epoch = datetime.datetime.utcfromtimestamp(0)
    return (dt - epoch).total_seconds() * 1000.0

userPrincipalsResponse = content.json()
tokenTimeStamp = userPrincipalsResponse['streamerInfo']['tokenTimestamp']
date = dateutil.parser.parse(tokenTimeStamp, ignoretz = True)
tokenTimeStampAsMs = unix_time_millis(date)

credentials = {
    "userid": userPrincipalsResponse['accounts'][0]['accountId'],
    "token": userPrincipalsResponse['streamerInfo']['token'],
    "company": userPrincipalsResponse['accounts'][0]['company'],
    "segment": userPrincipalsResponse['accounts'][0]['segment'],
    "cddomain": userPrincipalsResponse['accounts'][0]['accountCdDomainId'],
    "usergroup": userPrincipalsResponse['streamerInfo']['userGroup'],
    "accesslevel": userPrincipalsResponse['streamerInfo']['accessLevel'],
    "authorized": "Y",
    "timestamp": int(tokenTimeStampAsMs),
    "appid": userPrincipalsResponse['streamerInfo']['appId'],
    "acl": userPrincipalsResponse['streamerInfo']['acl']
}
login_request = {
    "requests": [
            {
                "service": "ADMIN",
                "command": "LOGIN",
                "requestid": '0',
                "account": userPrincipalsResponse['accounts'][0]['accountId'],
                "source": userPrincipalsResponse['streamerInfo']['appId'],
                "parameters": {
                    "credential": urllib.parse.urlencode(credentials),
                    "token": userPrincipalsResponse['streamerInfo']['token'],
                    "version": "1.0"
                }
            }
    ]
}
qos_request = {
    "requests": [
            {
                "service": "ADMIN",
                "requestid": '1',
                "command": "QOS",
                "account": userPrincipalsResponse['accounts'][0]['accountId'],
                "source": userPrincipalsResponse['streamerInfo']['appId'],
                "parameters": {
                    "qoslevel": '0'
                }
            }
    ]
}
quote_request = {
    "requests": [
            {
                "service": "QUOTE",
                "command": "SUBS",
                "requestid": '2',
                "account": userPrincipalsResponse['accounts'][0]['accountId'],
                "source": userPrincipalsResponse['streamerInfo']['appId'],
                "parameters": {
                    "keys": 'SPY, QQQ, AAPL, BA',
                    "fields": '0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16'
                }
            }
    ]
}
# encode requests
login = json.dumps(login_request)
qos = json.dumps(qos_request)
quote = json.dumps(quote_request)

In [6]:
# https://developer.tdameritrade.com/content/streaming-data#_Toc504640598
# print('If code executes then stops then you have an error in the code.')
# print('You will not get an error code.')
import pyodbc
import websockets
import asyncio
import datetime
import math
class StreamingClient():

    def __init__(self):
        self.conn = None
        self.cursor = None
    
    def DBconnect(self):
        server = 'DESKTOP-LBDSMI2'
        database = 'TDA'
        sql_driver = '{ODBC Driver 17 for SQL Server}'
        
        self.conn = pyodbc.connect( driver = sql_driver,
                                  server = server,
                                  database = database,
                                  trusted_connection = 'yes')
        self.cursor = self.conn.cursor()


    def insert( self, query, data_tuple):
        self.DBconnect()
        self.cursor.execute(query, data_tuple)
        self.conn.commit()
        self.conn.close()
        print(' ', end = '')
        

    def get_seconds( self, epoch):
        #epoch =  1593511118719 / 1000
        hours            = epoch % 86400            # Modding by seconds in a day  → Seconds since midnight 
        minutes          = hours % (60 * 60)        # Modding by seconds in a hour → Seconds since last hour
        seconds          = minutes % 60             # Modding by seconds in a minute → Seconds since last minute 
        #hour             = int( hours / 3600 )
        #minute           = int( minutes / 60 )
        #print ( '{}:{}:{}'.format( hour, minute, seconds ) )
        return seconds

    async def socketconnect(self):
        uri = 'wss://' + userPrincipalsResponse['streamerInfo']['streamerSocketUrl'] + '/ws'
        self.connection = await websockets.client.connect(uri)
        if self.connection.open:
            print('Connection established.  Client correctly connected.')
            return self.connection
    
    async def send(self, message):
        await self.connection.send(message)


    async def receive(self, connection):
        previous_price = 0
        previous_epoch = 0

        while True:
            try:
                
                message = await connection.recv()
                message_decoded = json.loads(message)
                if 'data' in message_decoded.keys():
                    data = message_decoded['data'][0]
                    if 'content' in data.keys():
                        content = data['content']
                        for i in range( len(content) ):
                            k = ['3', '10']
                            if all(k in content[i] for k in ('3', '10')):
                                timestamp           = data['timestamp']
                                Symbol              = content[i]['key']
                                BidPrice            = 0.0
                                AskPrice            = 0.0
                                LastPrice           = 0.0
                                BidSize             = 0
                                AskSize             = 0
                                AskID               = ''
                                BidID               = ''
                                TotalVolume         = 0
                                LastSize            = 0
                                QuoteTime           = 0
                                HighPrice           = 0.0 #content['12']
                                LowPrice            = 0.0 #content['13']
                                BidTick             = '' #content['14'] #  Need to add
                                ClosePrice          = 0 #content['15']
                                ExchangeID          = '' #content['16']

                                if '1' in content[i].keys():
                                    BidPrice        = content[i]['1']

                                if '2' in content[i].keys():
                                    AskPrice        = content[i]['2']
                            
                                LastPrice           = content[i]['3']
                                
                                if '4' in content[i].keys():
                                    BidSize         = content[i]['4']
                                
                                if '5' in content[i].keys():
                                    AskSize         = content[i]['5']
                        
                                if '6' in content[i].keys():
                                    AskID           = content[i]['6']
                                
                                if '7' in content[i].keys():
                                    BidID           = content[i]['7']

                                if '8' in content[i].keys():
                                    TotalVolume     = content[i]['8']

                                if '9' in content[i].keys():
                                    LastSize        = content[i]['9']

                                TradeTime           = content[i]['10']

                                if '11' in content[i].keys():
                                    QuoteTime       = content[i]['11']

                                if '12' in content[i].keys():
                                    HighPrice       = content[i]['12']

                                if '13' in content[i].keys():
                                    LowPrice        = content[i]['13']
                                
                                if '14' in content[i].keys():
                                    BidTick         = content[i]['14']

                                if '15' in content[i].keys():
                                    ClosePrice      = content[i]['15']

                                if '16' in content[i].keys():
                                    ExchangeID      = content[i]['16']
                                    print('_16_', end = '')
                                   
                                query = 'INSERT INTO datastream (timestamp, Symbol, BidPrice, AskPrice, LastPrice, BidSize, AskSize, AskID, BidID, TotalVolume, LastSize, TradeTime, QuoteTime, HighPrice, LowPrice, BidTick, ClosePrice, ExchangeID) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'
                                data_tuple = ( timestamp, Symbol, BidPrice, AskPrice, LastPrice, BidSize, AskSize, AskID, BidID, TotalVolume, LastSize, TradeTime, QuoteTime, HighPrice, LowPrice, BidTick, ClosePrice, ExchangeID )
                                self.insert(query, data_tuple)

                print('.', end = '')
                
            except websockets.exceptions.ConnectionClosed:
                print('Websocket connection is closed.')
                break


    async def heartbeat(self, connection):
        
        while True:
            try:
                await connection.send('ping')
                await asyncio.sleep(5)
            except websockets.exceptions.ConnectionClosed:
                print('Websocket connection is closed.')
                break

In [ ]:
# pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()

if __name__ == '__main__':
    client = StreamingClient()
    loop = asyncio.get_event_loop()
    socket = loop.run_until_complete(client.socketconnect())
    tasks = [asyncio.ensure_future(client.receive(socket)),
             
             asyncio.ensure_future(client.send(login)),
             asyncio.ensure_future(client.receive(socket)),

             asyncio.ensure_future(client.send(qos)),
             asyncio.ensure_future(client.receive(socket)),            
             
             asyncio.ensure_future(client.send(quote)),
             asyncio.ensure_future(client.receive(socket)),
            ]
    
    loop.run_until_complete(asyncio.wait(tasks))

Connection established.  Client correctly connected.
...._16_ _16_ _16_ _16_ .  .    . .  . . . .  .    ..  . .  .  . .  . .  . ..  .  .  .  ..    ..   ..   .  .  ..    .   . .  . .   ..    ...  .  .  . . .  .    ..   . .    . ..  .  . . .. .   ..   ..  .. . . . .  . .   .  .  . .. . . . .  ..   .  . .  ..  . .    .   . ..  .  ..  .. . . . .  ..  .  .  .    .. . .   ..    . .  . .  . . ..  .    ..  .  . .   . . . .  ..   . .  . .  .  . ..  . .  .  .  ..   .    .. .  .  ..   . .    .. . .  ..   . .   .  . . . . .   . ..